# **Tabulation**

## **Load Dataset & Depedencies**


In [2]:
import re
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from rich import print
from rich.table import Table
from rich.console import Console

## **Coding Kategori Level Risiko**

Diambil dari kolom `Risk_Level` dengan value berisi integer 1-5, kemudian dikategorikan berdasarkan pelabelan `BCM Institute`


In [37]:
data = pd.read_csv("csv_result-Final Dataset.csv")


def categorized_risk(df):
    bins = [1, 2, 3, 4, 5, np.inf]
    labels = [
        "(1) Very Low Risk",
        "(2) Low Risk",
        "(3) Medium Risk",
        "(4) High Risk",
        "(5) Very High Risk",
    ]
    df["Risk_Level_Category"] = pd.cut(
        df["Risk_Level"], bins=bins, labels=labels, right=False
    )
    return df


df = categorized_risk(df)

Risk_Level = pd.crosstab(
    index=df[df["Project"] == "Transaction Processing System"]["Risk_Level_Category"],
    columns="count",
)
Risk_Level["Percentage"] = (
    Risk_Level["count"] / Risk_Level["count"].sum() * 100
).round(2)
total_row = pd.DataFrame(
    {
        "count": [Risk_Level["count"].sum()],
        "Percentage": [Risk_Level["Percentage"].sum().round(1)],
    },
    index=["Total"],
)
Risk_Level = pd.concat([Risk_Level, total_row])
Risk_Level = Risk_Level.reset_index()
Risk_Level = Risk_Level.rename(columns={"index": "Risk Level"})
Risk_Level = Risk_Level[["Risk Level", "count", "Percentage"]]
display(Risk_Level)

,Risk Level,count,Percentage
0,(1) Very Low Risk,5,4.24
1,(2) Low Risk,40,33.90
2,(3) Medium Risk,41,34.75
3,(4) High Risk,23,19.49
4,(5) Very High Risk,9,7.63
5,Total,118,100.00


## **Tabel Tabulasi Level Risiko**

Create tables to see the relationship between two nominal variables, such as `Risk Level` with `Project Category` and `Requirement Type`. This can reveal, for example, if one project category consistently has a higher proportion of "high-risk" items.


In [32]:
Risk_Project = pd.crosstab(index=df["Project"], columns=df["Risk_Level_Category"])
Risk_Project["Total"] = Risk_Project.sum(axis=1).astype(int)
Risk_Project["Percentage"] = Risk_Project["Total"] / Risk_Project["Total"].sum() * 100
total_row = Risk_Project.sum(axis=0).to_frame().T
total_row.index = ["Total"]
Risk_Project = pd.concat([Risk_Project, total_row])
Risk_Project = Risk_Project.reset_index()
Risk_Project = Risk_Project.rename(columns={"index": "Project Type"})
# Risk_Project = Risk_Project.drop(columns=["Risk_Level"])
display(Risk_Project)

Risk_Level_Category,Project Type,(1) Very Low Risk,(2) Low Risk,(3) Medium Risk,(4) High Risk,(5) Very High Risk,Total,Percentage
0,Enterprise System,11.0,34.0,8.0,6.0,0.0,59.0,19.732441
1,Management Information System,4.0,34.0,26.0,16.0,7.0,87.0,29.096990
2,Safety Critical System,8.0,27.0,0.0,0.0,0.0,35.0,11.705686
3,Transaction Processing System,5.0,40.0,41.0,23.0,9.0,118.0,39.464883
4,Total,28.0,135.0,75.0,45.0,16.0,299.0,100.000000


In [41]:
Risk_Project = pd.crosstab(index=df["Requirement"], columns=df["Risk_Level_Category"])
Risk_Project["Total"] = Risk_Project.sum(axis=1).astype(int)
Risk_Project["Percentage"] = Risk_Project["Total"] / Risk_Project["Total"].sum() * 100
total_row = Risk_Project.sum(axis=0).to_frame().T
total_row.index = ["Total"]
Risk_Project = pd.concat([Risk_Project, total_row])
Risk_Project = Risk_Project.reset_index()
Risk_Project = Risk_Project.rename(columns={"index": "Requirement Type"})
display(Risk_Project)

Risk_Level_Category,Requirement Type,(1) Very Low Risk,(2) Low Risk,(3) Medium Risk,(4) High Risk,(5) Very High Risk,Total,Percentage
0,Constraints,1.0,7.0,8.0,2.0,2.0,20.0,6.688963
1,Functional,21.0,79.0,35.0,25.0,9.0,169.0,56.521739
2,Interfaces,1.0,19.0,11.0,6.0,0.0,37.0,12.374582
3,Performance,3.0,12.0,1.0,2.0,1.0,19.0,6.354515
4,Reliability & Availability,0.0,2.0,1.0,2.0,0.0,5.0,1.672241
5,Safety,0.0,3.0,0.0,0.0,0.0,3.0,1.003344
6,Security,2.0,7.0,9.0,2.0,1.0,21.0,7.023411
7,Standards,0.0,1.0,1.0,0.0,0.0,2.0,0.668896
8,Supportability,0.0,0.0,1.0,0.0,0.0,1.0,0.334448
9,Usability,0.0,5.0,8.0,6.0,3.0,22.0,7.357860
